In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2025-04-20 22:08:53.986553: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745186934.178690      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745186934.239748      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Configuration

In [2]:
DATA_DIR = '/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset'
IMG_SIZE = (128, 128)  # target size for CNN
NUM_CLASSES = 9
MODEL_PATH = 'fish_classifier.h5'

In [3]:
# mapping folder names to indices
CLASSES = sorted([d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))])
CLASS_TO_IDX = {cls: idx for idx, cls in enumerate(CLASSES)}

# Preprocessing & Segmentation

In [4]:
def preprocess_and_segment(img_path):
    # Load image
    img = cv2.imread(img_path)
    # Resize
    img_resized = cv2.resize(img, IMG_SIZE)
    # Convert to grayscale
    gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
    # Noise reduction
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    # Contrast adjustment via histogram equalization
    equalized = cv2.equalizeHist(blurred)
    # Thresholding for segmentation (Otsu)
    _, thresh = cv2.threshold(equalized, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # Morphological operations (closing to fill holes)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)
    # Find contours
    contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Create mask of largest contour (assumes fish is largest object)
    mask = np.zeros_like(gray)
    if contours:
        largest = max(contours, key=cv2.contourArea)
        cv2.drawContours(mask, [largest], -1, 255, -1)
    # Apply mask to original resized image
    masked = cv2.bitwise_and(img_resized, img_resized, mask=mask)
    # Normalize to [0,1]
    masked = masked.astype('float32') / 255.0
    return masked


# Data Loading

In [5]:
def load_dataset():
    images = []
    labels = []
    for cls in CLASSES:
        folder = os.path.join(DATA_DIR, cls, cls)
        for fname in os.listdir(folder):
            if not fname.lower().endswith('.png'):
                continue
            path = os.path.join(folder, fname)
            img = preprocess_and_segment(path)
            images.append(img)
            labels.append(CLASS_TO_IDX[cls])
    X = np.array(images)
    y = to_categorical(labels, num_classes=NUM_CLASSES)
    return train_test_split(X, y, test_size=0.2, random_state=42)

# Model Definition

In [6]:
def build_classifier(input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3), num_classes=NUM_CLASSES):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Training & Saving

In [7]:
def train_and_save():
    X_train, X_val, y_train, y_val = load_dataset()
    # Data augmentation
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True
    )
    datagen.fit(X_train)
    model = build_classifier()
    model.fit(datagen.flow(X_train, y_train, batch_size=32),
              epochs=20,
              validation_data=(X_val, y_val))
    model.save(MODEL_PATH)
    print(f"Model saved to {MODEL_PATH}")

In [8]:
def load_and_predict(img_path):
    model = load_model("/kaggle/working/fish_classifier.h5")
    img = preprocess_and_segment(img_path)
    img = np.expand_dims(img, axis=0)
    preds = model.predict(img)
    idx = np.argmax(preds, axis=1)[0]
    return CLASSES[idx], preds[0][idx]

In [10]:
train_and_save()

Epoch 1/20
225/225 ━━━━━━━━━━━━━━━━━━━━ 28s 104ms/step - accuracy: 0.2871 - loss: 1.8572 - val_accuracy: 0.6000 - val_loss: 1.0172
Epoch 2/20
225/225 ━━━━━━━━━━━━━━━━━━━━ 23s 100ms/step - accuracy: 0.5750 - loss: 1.1125 - val_accuracy: 0.7594 - val_loss: 0.6760
Epoch 3/20
225/225 ━━━━━━━━━━━━━━━━━━━━ 23s 100ms/step - accuracy: 0.6831 - loss: 0.8409 - val_accuracy: 0.8383 - val_loss: 0.4529
Epoch 4/20
225/225 ━━━━━━━━━━━━━━━━━━━━ 23s 100ms/step - accuracy: 0.7526 - loss: 0.6515 - val_accuracy: 0.8644 - val_loss: 0.3881
Epoch 5/20
225/225 ━━━━━━━━━━━━━━━━━━━━ 23s 101ms/step - accuracy: 0.8126 - loss: 0.5191 - val_accuracy: 0.9061 - val_loss: 0.2660
Epoch 6/20
225/225 ━━━━━━━━━━━━━━━━━━━━ 23s 100ms/step - accuracy: 0.8529 - loss: 0.4045 - val_accuracy: 0.9550 - val_loss: 0.1406
Epoch 7/20
225/225 ━━━━━━━━━━━━━━━━━━━━ 23s 100ms/step - accuracy: 0.8682 - loss: 0.3660 - val_accuracy: 0.9306 - val_loss: 0.2016
Epoch 8/20
225/225 ━━━━━━━━━━━━━━━━━━━━ 23s 101ms/step - accuracy: 0.9062 - loss: 0

In [13]:
print(load_and_predict("/kaggle/input/a-large-scale-fish-dataset/NA_Fish_Dataset/Horse Mackerel/00003.png"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 294ms/step
('Hourse Mackerel', 0.9895912)


In [16]:
print(CLASSES)
print(CLASS_TO_IDX)

['Black Sea Sprat', 'Gilt-Head Bream', 'Hourse Mackerel', 'Red Mullet', 'Red Sea Bream', 'Sea Bass', 'Shrimp', 'Striped Red Mullet', 'Trout']
{'Black Sea Sprat': 0, 'Gilt-Head Bream': 1, 'Hourse Mackerel': 2, 'Red Mullet': 3, 'Red Sea Bream': 4, 'Sea Bass': 5, 'Shrimp': 6, 'Striped Red Mullet': 7, 'Trout': 8}
